# Pixel intensity

Extracts respiration signals with the pixel intensity method. The pixel intensity method calculates the average pixel intensity within a region of interest (ROI) over time. The ROI is defined by a bounding box in the chest area of the subject. The bounding box is determined by the subjects face and shoulder positions in the first frame of the video.

In [ ]:
import os
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])
scenarios

In [ ]:
from datetime import datetime

manifest = {
    'timestamp_start': datetime.now(),
    'scenarios': scenarios,
}

In [ ]:
import cv2
import respiration.roi as roi
import respiration.utils as utils
import respiration.extractor.pixel_intensity as pixel_intensity

from tqdm.auto import tqdm

extracted_signals = []

for (subject, setting) in tqdm(scenarios):
    print(f'Processing {subject} - {setting}')

    frames, params = dataset.get_video_rgb(subject, setting)

    # Transform the first frame to grayscale
    grey_frame = cv2.cvtColor(frames[0], cv2.COLOR_RGB2GRAY)
    face = roi.detect_chest(grey_frame)

    # Extract the signal from the RGB frames
    pi_start = datetime.now()
    pi_signal_rgb = pixel_intensity.average_pixel_intensity_rgb(frames, roi=face)
    extracted_signals.append({
        'subject': subject,
        'setting': setting,
        'method': 'pixel_intensity_rgb',
        'duration': datetime.now() - pi_start,
        'sampling_rate': params.fps,
        'signal': pi_signal_rgb.tolist(),
    })

    # Extract the signal from the grayscale frames
    frames = utils.rgb_to_grey(frames)
    pi_start = datetime.now()
    pi_signal_rgb = pixel_intensity.average_pixel_intensity(frames, roi=face)
    extracted_signals.append({
        'subject': subject,
        'setting': setting,
        'method': 'pixel_intensity_grey',
        'duration': datetime.now() - pi_start,
        'sampling_rate': params.fps,
        'signal': pi_signal_rgb.tolist(),
    })

    # Garbage collect the frames
    del frames

In [ ]:
manifest['timestamp_finish'] = datetime.now()

In [ ]:
import pandas as pd

df = pd.DataFrame(extracted_signals)
df.head()

In [ ]:
import respiration.utils as utils

output_dir = utils.dir_path('outputs', 'signals', mkdir=True)

# Save the evaluation dataframe
csv_path = os.path.join(output_dir, 'pixel_intensity.csv')
df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(output_dir, 'pixel_intensity_manifest.json')
utils.write_json(json_path, manifest)